# EDA + Entrenamiento: Web Scraping → ML de Regresión (Books to Scrape)

Este cuaderno ejecuta:
- Scraping del sitio de prueba Books to Scrape para construir `data/dataset.csv`.
- Análisis exploratorio (EDA).
- Entrenamiento de modelos: RandomForest (baseline) y Red Neuronal Densa (Keras).
- Evaluación: R² y MAE, gráficas y artefactos en `/results` y `/models`.


In [ ]:
# Setup: imports y paths
import os, sys, json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Detectar raíz del proyecto (carpeta que contiene /src)
PROJ = Path.cwd()
if not (PROJ / 'src').exists() and (PROJ.name.lower() == 'notebooks'):
    PROJ = PROJ.parent
if not (PROJ / 'src').exists():
    print('Advertencia: no se encontró carpeta src junto al notebook. Usando cwd=', PROJ)

if str(PROJ) not in sys.path:
    sys.path.insert(0, str(PROJ))

from src.scraper import scrape_books
from src.train_model import train_and_evaluate
from src.evaluate import main as eval_main

print('Project root:', PROJ)
print('pandas', pd.__version__)
import sklearn, tensorflow as tf
print('scikit-learn', sklearn.__version__)
print('tensorflow', tf.__version__)


In [ ]:
# Scraping: construir dataset si no existe
DATA_CSV = PROJ / 'data' / 'dataset.csv'
DATA_CSV.parent.mkdir(parents=True, exist_ok=True)

if not DATA_CSV.exists():
    print('Iniciando scraping (puede tardar varios minutos para ~1000 items)...')
    df_scraped = scrape_books(max_items=None, min_delay=0.2, max_delay=0.5)
    df_scraped = df_scraped.dropna(subset=['price', 'rating', 'category']).reset_index(drop=True)
    df_scraped.to_csv(DATA_CSV, index=False)
    print('Dataset generado:', DATA_CSV, 'filas =', len(df_scraped))
else:
    print('Dataset ya existe:', DATA_CSV)


In [ ]:
# Cargar dataset y EDA rápida
import pandas as pd
import io

df = pd.read_csv(DATA_CSV)
print('Shape:', df.shape)
display(df.head(5))

# Info
buf = io.StringIO()
df.info(buf=buf)
print(buf.getvalue())

# Resumen estadístico
display(df.describe(include='all').T)

# Nulos
na_rate = df.isna().mean().sort_values(ascending=False)
display(na_rate.head(10))


In [ ]:
# Visualizaciones básicas
sns.set_theme(style='whitegrid')

# Distribución de precios
plt.figure(figsize=(7,4))
sns.histplot(df['price'], bins=30, kde=True)
plt.title('Distribución de precios')
plt.tight_layout()
plt.show()

# Conteo de ratings
plt.figure(figsize=(6,4))
sns.countplot(x='rating', data=df)
plt.title('Conteo de rating (1-5)')
plt.tight_layout()
plt.show()

# Precio vs rating
plt.figure(figsize=(6,4))
sns.stripplot(x='rating', y='price', data=df, size=3, alpha=0.5)
plt.title('Precio vs Rating')
plt.tight_layout()
plt.show()

# Boxplot por categoría (top 10 por frecuencia)
top_cats = df['category'].value_counts().head(10).index
plt.figure(figsize=(10,5))
sns.boxplot(x='category', y='price', data=df[df['category'].isin(top_cats)])
plt.xticks(rotation=40, ha='right')
plt.title('Precio por Categoría (Top 10)')
plt.tight_layout()
plt.show()

# Correlación de variables numéricas
num_cols = ['price', 'rating', 'availability', 'description_len', 'title_len', 'n_reviews', 'has_desc']
plt.figure(figsize=(7,5))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación (numéricas)')
plt.tight_layout()
plt.show()

In [ ]:
# Entrenamiento (baseline + NN)
metrics = train_and_evaluate(data_path=str(DATA_CSV), epochs=100, batch_size=32, rf_estimators=300)
metrics

In [ ]:
# Evaluación y gráficas
from IPython.display import Image, display

eval_main()

images = [
    'scatter_baseline.png',
    'scatter_nn.png',
    'residuals_baseline.png',
    'residuals_nn.png',
    'feature_importances_top20.png',
    'training_history.png'
]

for img in images:
    p = PROJ / 'results' / img
    if p.exists():
        display(Image(filename=str(p)))
    else:
        print('No encontrado:', p)
